<a href="https://colab.research.google.com/github/ash0411/fiw/blob/main/Copy_of_fiw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing libraries that are required and are not implicitly stored in the google colab
!pip install keras-vggface
# to remove some utils files that do not exit 
!pip show keras-vggface
!pip install keras_applications
filename = "/usr/local/lib/python3.7/dist-packages/keras_vggface/models.py"
text = open(filename).read()
# replacing the keras.engine.topology with tensorfolw.keras.utils as the keras.engine.topology 
# is not present in the tensorflow framework
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))

Name: keras-vggface
Version: 0.6
Summary: VGGFace implementation with Keras framework
Home-page: https://github.com/rcmalli/keras-vggface
Author: Refik Can MALLI
Author-email: mallir@itu.edu.tr
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, keras, six, h5py, scipy, pillow, pyyaml
Required-by: 
     |████████████████████████████████| 50 kB 2.9 MB/s 


20951

In [2]:
# testing for GPU 
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# importing basic libraries
import numpy as np
import pandas as pd
import os
from random import choice, sample
# importing visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
# importing library for reading image
from PIL import Image
from glob import glob
import tensorflow as tf
from collections import defaultdict
from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D,Flatten, Dense, Dropout, InputLayer,GlobalAvgPool2D, GlobalMaxPool2D, Concatenate, Multiply, Subtract, Add
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing import image
# libraries to build the  model
from keras import backend as K
K.set_image_data_format('channels_last')
import keras
from keras.models import Model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
import tensorflow as tf
import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [4]:
# setting the kaggle.json file and using it to import the dataset from kaggle.
!rmdir /root/.kaggle
!mkdir /root/.kaggle
!cp kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c recognizing-faces-in-the-wild

rmdir: failed to remove '/root/.kaggle': No such file or directory
  0% 0.00/354k [00:00<?, ?B/s]
100% 354k/354k [00:00<00:00, 53.9MB/s]
100% 130M/130M [00:00<00:00, 134MB/s]
100% 130M/130M [00:00<00:00, 145MB/s]
  0% 0.00/77.6k [00:00<?, ?B/s]
100% 77.6k/77.6k [00:00<00:00, 80.4MB/s]
  0% 0.00/33.7k [00:00<?, ?B/s]
100% 33.7k/33.7k [00:00<00:00, 30.7MB/s]
 81% 22.0M/27.1M [00:00<00:00, 57.3MB/s]
100% 27.1M/27.1M [00:00<00:00, 108MB/s] 
 86% 59.0M/68.6M [00:00<00:00, 93.5MB/s]
100% 68.6M/68.6M [00:00<00:00, 139MB/s] 
 77% 23.0M/29.7M [00:00<00:00, 65.3MB/s]
100% 29.7M/29.7M [00:00<00:00, 118MB/s] 
  0% 0.00/156k [00:00<?, ?B/s]
100% 156k/156k [00:00<00:00, 152MB/s]
 82% 28.0M/34.1M [00:00<00:00, 47.5MB/s]
100% 34.1M/34.1M [00:00<00:00, 76.8MB/s]
100% 111M/111M [00:00<00:00, 112MB/s]  
100% 111M/111M [00:01<00:00, 114MB/s]
  0% 0.00/1.47k [00:00<?, ?B/s]
100% 1.47k/1.47k [00:00<00:00, 1.53MB/s]
  0% 0.00/8.67k [00:00<?, ?B/s]
100% 8.67k/8.67k [00:00<00:00, 6.86MB/s]


In [20]:
# making the face directory to store the testing and training files
!mkdir face
# unzipping the files to extract the training and testing data
!unzip 'train.zip' -d '/content/face/train'
!unzip 'test.zip' -d '/content/face/test'

In [6]:
# reading relationship file to get insights on the relationships between different people
train_rel_df = pd.read_csv('train_relationships.csv')
train_rel_df.head()
train_file_path = "train_relationships.csv"
val_famillies = ["F09" ,'F08'] # setting validation families to those that are in 900's and 800's

In [7]:
# finding the number of families in training and putting them in a list
train_url =  '/content/face/train/'
family_dist = os.listdir(train_url)
print('the number of families are {}'.format(len(family_dist)))
print(family_dist[:10])

the number of families are 470
['F0258', 'F0432', 'F0733', 'F0313', 'F0415', 'F0745', 'F0399', 'F0845', 'F0334', 'F0290']


In [8]:
#finding the number of families in the test case and putting them in a list
test_url =  '/content/face/test/'
testing_pics = os.listdir(test_url)
print("the number of testing pictures are {}".format(len(testing_pics)))
print(testing_pics[:10])

the number of testing pictures are 6282
['face05782.jpg', 'face03590.jpg', 'face01606.jpg', 'face05957.jpg', 'face04964.jpg', 'face01879.jpg', 'face02129.jpg', 'face05050.jpg', 'face03472.jpg', 'face05665.jpg']


In [9]:
# reading the paths of all the images and seperating them into training and validation set
# the 800's and 900's are in validation and rest for training 
# seperating the destination of training and testing images
family_set_images = glob(train_url + "*/*/*.jpg")
training_images = [x for x in family_set_images if val_famillies[0] not in x and val_famillies[1] not in x]
validation_images = [x for x in family_set_images if val_famillies[0] in x or val_famillies[1] in x ]

In [10]:
# Mapping the family_member names with all the images of the member for training data
train_pti_mapping = defaultdict(list)
family_member = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in family_set_images]
training_set = [] # list that will contain all the families for the training set
for x in training_images:
    train_pti_mapping[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)
    if x.split('/')[-3] in training_set:
      continue
    else:
      training_set.append(x.split('/')[-3])
# Mapping the family_member names with all the images of the member for validation data
validation_pti_mapping = defaultdict(list)
validation_set = [] # list that will contain all the families for the validation set
for x in validation_images:
    validation_pti_mapping[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)
    if x.split('/')[-3]  in validation_set:
      continue
    else:
      validation_set.append(x.split('/')[-3])
  

In [11]:
#extracting the values from the relationships csv and converting them to tuples in a list
train_rel_df = list(zip(train_rel_df.p1.values, train_rel_df.p2.values))
# checking if th family_member is present in the training dataset or not. 
train_rel_df = [x for x in train_rel_df if x[0] in family_member and x[1] in family_member]

In [12]:
# seperating into training and validation dataset list
train = [x for x in train_rel_df if val_famillies[0] not in x[0] and val_famillies[1] not in x[0]]
val = [x for x in train_rel_df if val_famillies[0] in x[0] or val_famillies[1] in x[0]]

In [13]:
print('the training dataset contains {} number of images'.format(len(train)))
print('the validation dataset contains {} number of images'.format(len(val)))

the training dataset contains 2692 number of images
the validation dataset contains 670 number of images


In [14]:
# checking if families starting with 'F09' or 'F08' are present in the training data
# Families starting with 'F09' or 'F08' are present in only validation dataset
for x in train:
  if 'F09' in x[0]:
    print(x) 
  if 'F08' in x[0]:
    print(x)

In [15]:
# A helper function to scan the images from the drive
def img_scan(path):
    img = image.load_img(path, target_size=(224, 224))
    img = np.array(img).astype(np.float)#reshape(3,224,224)
    return img

In [16]:
# An image generation function
# It randomly selects 32 image pairs 16 being kin and 16 not
# it is called during the fitting of th emodel
def image_generation(dataset, pti_map, batch_size=32):
    family_member = list(pti_map.keys())
    while True:
        batches = sample(dataset, batch_size // 2)
        labels = [1] * len(batches)
        while len(batches) < batch_size:
            family_member1 = choice(family_member)
            family_member2 = choice(family_member)
        # to append same no of relations that are in the table to no of relations not in the relations
        # to make the data have same proportion of negative and postive examples

            if family_member1 != family_member2 and (family_member1, family_member2) not in dataset and (family_member1, family_member2) not in dataset:
                batches.append((family_member1, family_member2))
                labels.append(0)

        for x in batches:
            if not len(pti_map[x[0]]):
                print(x[0])
        # transfering the first set of images in list X1
        # selecting random images for a particular person from the list 
        X1 = [choice(pti_map[x[0]]) for x in batches]
        X1 = np.array([img_scan(x) for x in X1])
        # transfering the second set of images in list X2
        # selecting random sets of images for a particular person in the list
        X2 = [choice(pti_map[x[1]]) for x in batches]
        X2 = np.array([img_scan(x) for x in X2])
        yield [np.array(X1), np.array(X2)], np.array(labels)

In [17]:
# custom checking if the image generation function is working
temp = image_generation(train,train_pti_mapping)

In [18]:
# loading the VGGFACE model and examining its structure
# instead of resnet50 we have also tried senet50 on the dataset as a base model for VGGFace
base_model = VGGFace(model = 'resnet50', include_top = False)
base_model.summary()

94707712/94694792 [==============================] - 4s 0us/step
Model: "vggface_resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, None, None,   9408        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, None, None,   256        ['conv1/7x7_s2[0][0]']           
 ation)           

In [19]:
# struture of Functional keras model
def vgg_face_resnet50_model():
    input_1 = Input(shape=(224,224,3))
    input_2 = Input(shape=(224,224,3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1 = Concatenate(axis=-1)([GlobalAvgPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalAvgPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    
    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4  = Subtract()([x1_, x2_])    
    x   = Concatenate(axis=-1)([x4, x3])
    x   = Dense(256, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = BatchNormalization()(x)
    x   = Dense(256, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = Dropout(0.01)(x) 
    x   = Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = BatchNormalization()(x)
    x   = Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = Dropout(0.01)(x)    
    x   = Dense(100, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)    
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))
    model.summary()

    return model

In [20]:
# saving path for model
file_path = "/drive/MyDrive/vgg_face.h5"
# to save model if validation accuracy increases
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')
# to stop model incase only the validation accuracy is increasing 
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, 
                          patience=15, verbose=0, mode='auto')
# to reduce the learning rate in case the validation accuracy has stop improving
reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.2, 
                                      patience=10, verbose=1)

callbacks_list = [checkpoint,early_stop]
#compiling the model
training_model = vgg_face_resnet50_model()
training_model_hist=training_model.fit(image_generation(train, train_pti_mapping, batch_size=32), 
                            use_multiprocessing=True,
                    validation_data=image_generation(val, validation_pti_mapping, batch_size=32), 
                            epochs=10, 
                            verbose=1,workers = 4, 
                            callbacks=callbacks_list,
                            steps_per_epoch=200,
                            validation_steps=100)